In [ ]:
import taichi as ti
import taichi_three as t3

In [ ]:
import numpy as np
from PIL import Image

from pydrake.common.eigen_geometry import Quaternion

In [ ]:
full_state0 = np.load("full_state0.npy", allow_pickle=True).item()
full_state0_img = Image.open("full_state0.png")
x, _v, _F, _C, primitive_states = full_state0["state"]
cyl_p = primitive_states[:3]
cyl_qwxyz = primitive_states[3:]
cyl_R = Quaternion(cyl_qwxyz).rotation().copy()
display(full_state0_img)

In [ ]:
ti.init(ti.gpu)

In [ ]:
pos = ti.Vector.field(3, ti.f32, len(x))
pos.from_numpy(x)

In [ ]:
scene = t3.Scene()

# TODO(eric): How to add color to particles?
particles = t3.ScatterModel(radius=4)
particles.particles = pos
scene.add_model(particles)

# Lazy textured cylinder + scaling.
obj = t3.readobj("textured_cylinder.obj")
texture = ti.imread("textured_cylinder.png")
obj["vp"] = obj["vp"][:, :3]  # dunno why this is 6-elem; some weird export from MeshLab?
r = 0.0254
h = 0.254
obj["vp"] *= np.array([[r, h, r]])
cylinder = t3.Model.from_obj(obj, texture)
scene.add_model(cylinder)

# need to pass in ndarray
camera = t3.Camera(
    pos=[0.5, 0.8 , 1.5],
    target=x.mean(axis=0).tolist(),
)
scene.add_camera(camera)

light = t3.Light()
scene.add_light(light)

In [ ]:
# N.B. For whatever reason, we have to call `scene.render()` to
# ensure we can change the cylinder pose.
scene.render()

In [ ]:
cylinder.L2W.offset.from_numpy(cyl_p)
cylinder.L2W.matrix.from_numpy(cyl_R)

In [ ]:
gui = ti.GUI("test", camera.res)
while gui.running:
    gui.get_event(None)
    gui.running = not gui.is_pressed(ti.GUI.ESCAPE)
    camera.from_mouse(gui)
    scene.render()
    gui.set_image(camera.img)
    gui.show()

In [ ]:
camera.pos.to_numpy()